In [ ]:
# Oanda API teszter

In [ ]:
## Account

In [1]:
import cfg

running price stream
running transaction stream
running account update polling


In [2]:
cfg.account

In [3]:
for t in cfg.account.trades:
    print(f'{t.currentUnits:>7.0f}, {t.instrument}, {t.unrealizedPL:>8.4f}, {t.openTime}')

    364, EUR_USD,  -0.1856, 2022-01-21T20:27:09.741023143Z
    375, EUR_GBP,   0.0000, 2022-01-21T16:24:06.360822096Z
   -372, USD_CHF,  -0.0287, 2022-01-21T21:48:11.417108389Z
    361, USD_JPY,  -0.1085, 2022-01-21T17:20:49.273744047Z
   -364, GBP_USD,   0.3567, 2022-01-21T18:54:08.365353679Z
   -375, AUD_USD,   0.5438, 2022-01-21T16:24:06.360938717Z
   -368, AUD_CAD,  -0.7173, 2022-01-21T21:54:11.568003098Z


In [4]:
print(f'{cfg.account.balance} nav:{cfg.account.NAV} margin:{cfg.account.marginAvailable}')

1537.6708 nav:1537.5312 margin:1463.6066


In [5]:
def check_breakeven_for_instrument(instrument: str, trades: list):
    all_breakeven = []
    for t in trades:
        if t.instrument == instrument:
            for o in cfg.account.orders:
                if o.id == t.stopLossOrderID:
                    all_breakeven.append(
                    (t.currentUnits > 0 ) and (o.price >=t.price)
                    or
                    (t.currentUnits <0 and o.price<=t.price))
    return all(all_breakeven)

In [6]:
check_breakeven_for_instrument('AUD_USD', cfg.account.trades)

False

In [7]:
for t in cfg.account.trades:
    print(t)
    break

id: '180469'
instrument: EUR_USD
price: 1.13439
openTime: '2022-01-21T20:27:09.741023143Z'
state: OPEN
initialUnits: 364.0
initialMarginRequired: 8.2578
currentUnits: 364.0
realizedPL: 0.0
unrealizedPL: -0.1856
marginUsed: 8.2583
financing: -0.0192
takeProfitOrderID: '180470'
stopLossOrderID: '180471'
trailingStopLossOrderID: '180472'


In [8]:
for o in cfg.account.orders:
    print(o)
    break

id: '180365'
createTime: '2022-01-21T16:24:06.360822096Z'
state: PENDING
type: TAKE_PROFIT
tradeID: '180364'
price: 0.84139
timeInForce: GTC
triggerCondition: DEFAULT


In [9]:
for t in cfg.account.trades:
    print(f'{t.instrument} {t.unrealizedPL}')

EUR_USD -0.1856
EUR_GBP 0.0
USD_CHF -0.0287
USD_JPY -0.1085
GBP_USD 0.3567
AUD_USD 0.5438
AUD_CAD -0.7173


In [10]:
for p in cfg.account.positions:
    print(p)
    break

instrument: EUR_USD
pl: -384.7528
unrealizedPL: -0.1856
marginUsed: 8.2583
resettablePL: -384.7528
financing: -78.4564
commission: 0.0
guaranteedExecutionFees: 0.0
long:
  units: 364.0
  averagePrice: 1.13439
  tradeIDs:
  - '180469'
  pl: -153.9157
  unrealizedPL: -0.1856
  resettablePL: -153.9157
  financing: -79.6147
  guaranteedExecutionFees: 0.0
short:
  units: 0.0
  pl: -230.8371
  unrealizedPL: 0.0
  resettablePL: -230.8371
  financing: 1.1583
  guaranteedExecutionFees: 0.0


In [11]:
cfg.instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.13388,
 'ask': 1.13488,
 'spread': 0.001}